In [1]:
# Re-imports after kernel reset
import os
import time
import json
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [2]:
# ---------------------------
# Aryabhata model setup
# ---------------------------
# model_id = "PhysicsWallahAI/Aryabhata-1.0"
# model_id = "Qwen/Qwen2.5-0.5B"
model_id = "MiniLLM/Pretrain-Qwen-200M"
tokenizer_aryabhata = AutoTokenizer.from_pretrained(model_id)
model_aryabhata = AutoModelForCausalLM.from_pretrained(model_id).to("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/814M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [7]:
# ---------------------------
# Stop strings and cleaner
# ---------------------------
stop_strings = ["<|im_end|>", "<|end|>", "<im_start|>", "⁠```python\n", "⁠<|im_start|>", "]}}]}}]"]

def strip_bad_tokens(s, stop_strings):
    for suffix in stop_strings:
        if s.endswith(suffix):
            return s[:-len(suffix)]
    return s

generation_config = GenerationConfig(
    max_new_tokens=4096,
    stop_strings=stop_strings
)

In [8]:
# ---------------------------
# Prompt formatter (matches website)
# ---------------------------
def create_chat_prompt_options(question, options):
    if options:
        opt_str = '\n'.join([f"{chr(65+i)}. {opt}" for i, opt in enumerate(options)])
        content = f"{question}\n\nOptions:\n{opt_str}"
    else:
        content = question
    return [
        {'role': 'system', 'content': 'Think step-by-step; put only the final answer inside \\boxed{}.'},
        {'role': 'user', 'content': content}
    ]


In [9]:
# ---------------------------
# Prompt formatter (matches website)
# ---------------------------
def create_chat_prompt_question(question):

    content = f"{question}"
    return [
        {'role': 'system', 'content': 'Think step-by-step; put only the final answer inside \\boxed{}.'},
        {'role': 'user', 'content': content}
    ]

In [10]:
# ---------------------------
# Prompt formatter (matches website)
# ---------------------------
def create_chat_prompt_system_mes_modified(question):

    content = f"{question}"
    return [
        {'role': 'system', 'content': 'Think step-by-step to come up to a final answer.'},
        {'role': 'user', 'content': content}
    ]

In [11]:
# ---------------------------
# Inference function
# ---------------------------
def run_aryabhata_prompt(messages, tokenizer, model):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, generation_config=generation_config, tokenizer=tokenizer)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [12]:
def return_assistant_part(tokenizer, outputs):
  # Return ONLY the assistant's part of the response (after last <|im_start|>assistant)
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "<|im_start|>assistant" in raw:
        return raw.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
    else:
        return raw.strip()

In [13]:
# ---------------------------
# Main processing function
# ---------------------------

from datetime import datetime

def process_all_json_files(input_folder, output_folder, list_json=None, only_question=False, with_options= False, w_sys_prompt_mod=False, local=False):
    os.makedirs(output_folder, exist_ok=True)
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Build dynamic suffix based on active flags
    suffix_parts = []
    if only_question:
        suffix_parts.append("only_question")
    if with_options:
        suffix_parts.append("with_options")
    if w_sys_prompt_mod:
        suffix_parts.append("w_sys_prompt_mod")
    suffix_str = "_".join(suffix_parts) if suffix_parts else "default"

    # Create subfolder with timestamp + mode suffix
    subfolder = os.path.join(output_folder, f"{suffix_str}_{timestamp_str}")
    os.makedirs(subfolder, exist_ok=True)

    if local:
      os.makedirs("./output", exist_ok=True)
      local_subfolder = os.path.join("./output", f"{suffix_str}_{timestamp_str}")
      os.makedirs(local_subfolder, exist_ok=True)

    if list_json is not None:
      input_files = list_json
    else:
      input_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]


    for filename in input_files:
        input_path = os.path.join(input_folder, filename)

        # Final output path
        output_path = os.path.join(subfolder, f"pred_{suffix_str}_{timestamp_str}_{filename}")


        with open(input_path, "r") as f:
            questions = json.load(f)

        results = []
        total_time_taken = 0

        for i, q in enumerate(tqdm(questions, desc=f"Processing {filename}")):
            question_text = q["question"]
            options = q.get("options", [])
            answer = q["answer"]

            messages_options = create_chat_prompt_options(question_text, options)
            messages_question = create_chat_prompt_question(question_text)
            message_system_mes_modified = create_chat_prompt_system_mes_modified(question_text)

            if with_options:
                start_time = time.time()
                response_options = run_aryabhata_prompt(messages_options, tokenizer_aryabhata, model_aryabhata)
                end_time = time.time()
                elapsed = end_time - start_time
                total_time_taken += elapsed

                print(f"→ [{i+1}/{len(questions)}] Time taken: {elapsed:.2f} sec")


                # Save response with original question
                q["aryabhata_response_Woptions"] = response_options
                q["aryabhata_response_Woptions"] = response_options
                q["inference_time_seconds_Woptions"] = round(elapsed, 2)

            if only_question:

                start_time = time.time()
                response_question = run_aryabhata_prompt(messages_question, tokenizer_aryabhata, model_aryabhata)
                end_time = time.time()
                elapsed = end_time - start_time
                total_time_taken += elapsed

                print(f"→ [{i+1}/{len(questions)}] Time taken: {elapsed:.2f} sec")

                # Save response with original question
                q["aryabhata_response_Wquestion"] = response_question
                q["inference_time_seconds_Wquestion"] = round(elapsed, 2)

            if w_sys_prompt_mod:

                start_time = time.time()
                response_sys_mes_modified = run_aryabhata_prompt(message_system_mes_modified, tokenizer_aryabhata, model_aryabhata)
                end_time = time.time()
                elapsed = end_time - start_time
                total_time_taken += elapsed

                print(f"→ [{i+1}/{len(questions)}] Time taken: {elapsed:.2f} sec")

                # Save response with original question
                q["aryabhata_response_sys_mes_modified"] = response_sys_mes_modified
                q["inference_time_seconds_sys_mes_modified"] = round(elapsed, 2)

            results.append(q)

            # ✅ Save incrementally and ensure Drive sync
            with open(output_path, "w") as fout:
                json.dump(results, fout, indent=2, ensure_ascii=False)
                fout.flush()
                os.fsync(fout.fileno())
            if local:
              # ✅ Save incrementally and ensure Drive sync
              with open(os.path.join(local_subfolder, f"pred_{suffix_str}_{timestamp_str}_{filename}"), "w") as fout:
                  json.dump(results, fout, indent=2, ensure_ascii=False)
                  fout.flush()
                  os.fsync(fout.fileno())



        print(f"\n✅ Total Time taken for {filename}: {total_time_taken:.2f} sec ({total_time_taken/60:.2f} min)")



In [14]:
# !mkdir input

In [15]:
# !mkdir output

In [ ]:
# process_all_json_files("./input", "./output", only_question=True, with_options= False, w_sys_prompt_mod=False)

In [16]:
def gen_response(input_folder, output_folder, list_json=None, only_question=False, with_options= False, w_sys_prompt_mod=False, local=False):
    os.makedirs(output_folder, exist_ok=True)

    total = 0
    for ques_paper in list_json:
      with open(os.path.join(input_folder, ques_paper), "r") as f:
        data = json.load(f)
      total += len(data)
      print(f"total question in {ques_paper}: ", len(data))
    print("total questions to gen response: ", total)

    process_all_json_files(input_folder, output_folder, list_json=list_json, only_question=only_question, with_options= with_options, w_sys_prompt_mod=w_sys_prompt_mod, local= local)




In [17]:
input_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/question_papers"
output_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/aryabhata1_gen_answers"

Jee_questions = sorted([f for f in os.listdir(input_folder) if f.endswith(".json")])
list_json = Jee_questions[:-2]
Jee_questions

['jan_22_shift_1_questions.json',
 'jan_22_shift_2_questions.json',
 'jan_23_shift_1_questions.json',
 'jan_23_shift_2_questions.json',
 'jan_24_shift_1_questions.json',
 'jan_24_shift_2_questions.json',
 'jan_28_shift_1_questions.json',
 'jan_28_shift_2_questions.json',
 'jan_29_shift_1_questions.json',
 'jan_29_shift_2_questions.json',
 'master_jee2025_maths_questions.json',
 'question_papers.json']

In [ ]:
list_json = Jee_questions[:-2]

list_json

['jan_22_shift_1_questions.json',
 'jan_22_shift_2_questions.json',
 'jan_23_shift_1_questions.json',
 'jan_23_shift_2_questions.json',
 'jan_24_shift_1_questions.json',
 'jan_24_shift_2_questions.json',
 'jan_28_shift_1_questions.json',
 'jan_28_shift_2_questions.json',
 'jan_29_shift_1_questions.json',
 'jan_29_shift_2_questions.json']

### defalut mode for testing

In [18]:
gen_response(input_folder, output_folder, list_json=list_json, only_question=False, with_options= False, w_sys_prompt_mod=False, local=True)

total question in jan_22_shift_1_questions.json:  25
total question in jan_22_shift_2_questions.json:  25
total question in jan_23_shift_1_questions.json:  25
total question in jan_23_shift_2_questions.json:  25
total question in jan_24_shift_1_questions.json:  25
total question in jan_24_shift_2_questions.json:  25
total question in jan_28_shift_1_questions.json:  25
total question in jan_28_shift_2_questions.json:  25
total question in jan_29_shift_1_questions.json:  25
total question in jan_29_shift_2_questions.json:  25
total questions to gen response:  250


Processing jan_22_shift_1_questions.json: 100%|██████████| 25/25 [00:00<00:00, 103.29it/s]



✅ Total Time taken for jan_22_shift_1_questions.json: 0.00 sec (0.00 min)


Processing jan_22_shift_2_questions.json: 100%|██████████| 25/25 [00:00<00:00, 116.57it/s]



✅ Total Time taken for jan_22_shift_2_questions.json: 0.00 sec (0.00 min)


...


✅ Total Time taken for jan_28_shift_2_questions.json: 0.00 sec (0.00 min)


Processing jan_29_shift_1_questions.json: 100%|██████████| 25/25 [00:00<00:00, 110.50it/s]



✅ Total Time taken for jan_29_shift_1_questions.json: 0.00 sec (0.00 min)


Processing jan_29_shift_2_questions.json: 100%|██████████| 25/25 [00:00<00:00, 116.82it/s]


✅ Total Time taken for jan_29_shift_2_questions.json: 0.00 sec (0.00 min)


# prod mod:

```python
gen_response(input_folder, output_folder, list_json=list_json, only_question=True, with_options= False, w_sys_prompt_mod=False, local=True)
```

In [ ]:
input_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/question_papers"
output_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/aryabhata1_Oques_gen_answers"

Jee_questions = sorted([f for f in os.listdir(input_folder) if f.endswith(".json")])
list_json = Jee_questions[:-2]

gen_response(input_folder, output_folder, list_json=list_json, only_question=True, with_options= False, w_sys_prompt_mod=False, local=True)

# prod mod:

```python
gen_response(input_folder, output_folder, list_json=list_json, only_question=False, with_options= True, w_sys_prompt_mod=False, local=True)
```

In [ ]:
input_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/question_papers"
output_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/aryabhata1_Woptions_gen_answers"

Jee_questions = sorted([f for f in os.listdir(input_folder) if f.endswith(".json")])
list_json = Jee_questions[:-2]

gen_response(input_folder, output_folder, list_json=list_json, only_question=False, with_options= True, w_sys_prompt_mod=False, local=True)

# prod mod:

```python
gen_response(input_folder, output_folder, list_json=list_json, only_question=False, with_options= False, w_sys_prompt_mod=True, local=True)
```

In [ ]:
input_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/question_papers"
output_folder = "/content/drive/MyDrive/Aryabhata_Eval/Jee-questions-maths-2025/aryabhata1_w_sys_prmt_gen_answers"

Jee_questions = sorted([f for f in os.listdir(input_folder) if f.endswith(".json")])
list_json = Jee_questions[:-2]

gen_response(input_folder, output_folder, list_json=list_json, only_question=False, with_options= False, w_sys_prompt_mod=True, local=True)

In [ ]:
# !rm -rf /content/output/default_20250727_100125
